In [2]:
# Load new atlas
import os
from os.path import join as pjoin
import numpy as np
# from dipy.io.streamline import load_trk

fname = 'hcp842_80_atlas.npz'
f = np.load(fname, allow_pickle=1)
atlas = f['atlas']
labels = f['labels']
label_names = f['label_names']
hierarchy = f['hierarchy']

fname = 'hcp842_80_centroids.npz'
f = np.load(fname, allow_pickle=1)
centroids = f['atlas']
clabels = f['labels']

In [3]:
label_names

array(['', 'AC', 'AF_L', 'AF_R', 'AR_L', 'AR_R', 'AST_L', 'AST_R', 'CB_L',
       'CB_R', 'CC', 'CC_ForcepsMajor', 'CC_ForcepsMinor', 'CC_Mid',
       'CNIII_L', 'CNIII_R', 'CNII_L', 'CNII_R', 'CNIV_L', 'CNIV_R',
       'CNVIII_L', 'CNVIII_R', 'CNVII_L', 'CNVII_R', 'CNV_L', 'CNV_R',
       'CST_L', 'CST_R', 'CS_L', 'CS_R', 'CTT_L', 'CTT_R', 'CT_L', 'CT_R',
       'C_L', 'C_R', 'DLF_L', 'DLF_R', 'EMC_L', 'EMC_R', 'FPT_L', 'FPT_R',
       'F_L_R', 'ICP_L', 'ICP_R', 'IFOF_L', 'IFOF_R', 'ILF_L', 'ILF_R',
       'LL_L', 'LL_R', 'MCP', 'MLF_L', 'MLF_R', 'ML_L', 'ML_R', 'MdLF_L',
       'MdLF_R', 'OPT_L', 'OPT_R', 'OR_L', 'OR_R', 'PC', 'PPT_L', 'PPT_R',
       'RST_L', 'RST_R', 'SCP', 'SLF_L', 'SLF_R', 'STT_L', 'STT_R',
       'TPT_L', 'TPT_R', 'UF_L', 'UF_R', 'V', 'VOF_L', 'VOF_R'],
      dtype='<U15')

In [4]:
#centroids = centroids.reshape((-1, 5, 2))

In [5]:
from fury import actor, window

scene = window.Scene()

L = label_names.tolist().index('MCP')
lines = atlas[labels==L]
print(len(lines))
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[1,1,0]]*len(lines),
                         linewidth=3, fake_tube=True, depth_cue=True,
                         opacity = 0.5))

L = label_names.tolist().index('EMC_L')
lines = atlas[labels==L]
print(len(lines))
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[0,1,1]]*len(lines),
                         linewidth=3, fake_tube=True, depth_cue=True,
                         opacity = 0.5))
    
lines = atlas[::30]
print(len(lines))
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[1,1,1]]*len(lines),
                         linewidth=3, fake_tube=True, depth_cue=True,
                         opacity = 0.1))
    
# L = label_names.tolist().index('CC_Mid')
# lines = atlas[labels==L]
# print(len(lines))
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[[1,1,0]]*len(lines),
#                          linewidth=3, fake_tube=True, depth_cue=True,
#                          opacity = 0.5))

    
# L = label_names.tolist().index('CC')
# lines = atlas[labels==L]
# print(len(lines))
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[[0,1,0]]*len(lines),
#                          linewidth=3, fake_tube=True, depth_cue=True,
#                          opacity = 0.5))
    
# lines = final_fibers[centroid_ixs]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[0,1,0], opacity = 0.5))

# for i in range(1, len(label_names)):
#     if '_R' in label_names[i]:
#         lines = final_fibers[final_labels==i]
#         color = np.random.rand(3)
#         #color /= np.linalg.norm(color)
#         if len(lines)>0:
#             scene.add(actor.line(lines, colors=color, opacity = 0.5))

window.show(scene)

1690
1642
4822


In [ ]:
from collections import Counter
{label_names[k]: v for k, v in Counter(clabels).items()}

In [ ]:
# from catboost import CatBoostClassifier
# model = CatBoostClassifier(task_type='GPU')
# model.load_model('atlas.cbm')

In [ ]:
from classibundler import classify
from dipy.tracking.streamline import set_number_of_points
NF = 5  #len(model.feature_names_) // 3
fibers = np.reshape(set_number_of_points([*atlas], NF), (-1, NF*3))
#prediction = model.predict(data).flatten()
prediction, is_reversed = classify(
    fibers, centroids, clabels, threshold=30)

In [ ]:
# Check model accuracey vs model size
# N = np.exp(np.arange(30, 101) * (np.log(5000)/100)).astype(int)[::-1]
# score_nocc, score_all = [], []

# for n in N:
#     if n < model.tree_count_:
#         model.shrink(n)
#     prediction = model.predict(data).flatten()
#     ixs = labels!=10
#     score_nocc.append(np.mean(prediction[ixs]==labels[ixs]))
#     score_all.append(np.mean(prediction==labels))

# import matplotlib.pyplot as plt
# plt.semilogx(N, 1-np.array(score_nocc))
# plt.semilogx(N, 1-np.array(score_all))
# plt.ylim([0, 0.02])
# plt.grid()

# model.shrink(1024)
# model.save_model('atlas.cbm')

In [ ]:
print('label %iou     gt  dtctd  correct')
NL = len(label_names)
for l in range(NL):
    detected = np.sum(prediction==l)
    correct = np.sum((prediction==l) & (labels==l))
    gt = np.sum(labels==l)
    iou = correct / (detected + gt - correct)
    print(f'{l:2}   {iou:.3f}  {gt:5}  {detected:5}  {correct:5}  {label_names[l]}')

In [ ]:
from collections import Counter
l = label_names.tolist().index('AST_L')
cnt_gt = Counter(prediction[labels==l])
cnt_rc = Counter(labels[prediction==l])
print('Predicted:', {label_names[k]: v for k, v in cnt_gt.most_common(10)})
print('Selected: ', {label_names[k]: v for k, v in cnt_rc.most_common(10)})

In [ ]:
from scipy.spatial.distance import cdist, pdist
AST_L = label_names.tolist().index('AST_L')
AST_L_ixs = np.where(labels==AST_L)[0]
z_ixs = np.where(labels==0)
A = pdist(fibers[AST_L_ixs].reshape((-1, 15)))
Z = pdist(fibers[z_ixs].reshape((-1, 15)))
AZ = cdist(fibers[AST_L_ixs].reshape((-1, 15)),
           fibers[z_ixs].reshape((-1, 15)))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 15))
plt.hist(AZ.flatten(), bins=300, histtype='step')
# plt.hist(Z.flatten(), bins=100, histtype='step')
plt.hist(A.flatten(), bins=100, histtype='step')
plt.grid()

In [ ]:
def export_as_obj(fibers, l_shift=0):
    strings = [
        f'v {v[0]:.6g} {v[1]:.6g} {v[2]:.6g}'
        for f in fibers
        for v in f
    ]
    i = 1 + l_shift
    for f in fibers:
        strings.append('l ' + ' '.join(map(str, range(i, i+len(f)))))
        i += len(f)
    return strings

In [ ]:
parts = []
l_shift = 0

for name in ['', 'UF_L', 'UF_R', 'IFOF_L', 'IFOF_R']:
    parts.append(f'o {name}')
    L = label_names.tolist().index(name)
    ixs = centroid_ixs[labels[centroid_ixs] == L]
#     A = fibers[ixs].reshape((-1, 5, 3))
    A = atlas[ixs]
    parts.extend(export_as_obj(A, l_shift))
    parts.append(f'\n')
    l_shift += sum(map(len, A))
    print(name, fibers[ixs].reshape((-1, 5, 3)).mean(1).mean(0))

with open('fibers.obj', 'wt') as f:
    f.write('\n'.join(parts))

In [ ]:
from fury import actor, window

scene = window.Scene()

#scene.add(actor.line(atlas[::10], colors=[1]*3, opacity = 0.05))

ixs = centroid_ixs[labels[centroid_ixs] == 0]
# lines = fibers[ixs].reshape((-1, 5, 3))
lines = atlas[ixs]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[1,1,1]]*len(lines), opacity = 0.5))

L = label_names.tolist().index('UF_L')
ixs = centroid_ixs[labels[centroid_ixs] == L]
lines = atlas[ixs]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[0,1,0]]*len(lines), opacity = 0.5))

L = label_names.tolist().index('UF_R')
ixs = centroid_ixs[labels[centroid_ixs] == L]
lines = atlas[ixs]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[0,.5,1]]*len(lines), opacity = 0.5))
    
L = label_names.tolist().index('IFOF_R')
ixs = centroid_ixs[labels[centroid_ixs] == L]
lines = atlas[ixs]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[1,0,1]]*len(lines), opacity = 0.5))
    
L = label_names.tolist().index('IFOF_L')
ixs = centroid_ixs[labels[centroid_ixs] == L]
lines = atlas[ixs]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[[1,.5,0]]*len(lines), opacity = 0.5))

# lines = final_fibers[centroid_ixs]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[0,1,0], opacity = 0.5))

# for i in range(1, len(label_names)):
#     if '_R' in label_names[i]:
#         lines = final_fibers[final_labels==i]
#         color = np.random.rand(3)
#         #color /= np.linalg.norm(color)
#         if len(lines)>0:
#             scene.add(actor.line(lines, colors=color, opacity = 0.5))

window.show(scene)

In [ ]:
from fury import actor, window

scene = window.Scene()

#scene.add(actor.line(atlas[::10], colors=[1]*3, opacity = 0.05))

i = label_names.tolist().index('UF_L')

lines = atlas[(labels==i) & (prediction!=i)]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[1, 0, 0], opacity = 0.5))

lines = atlas[(labels==i) & (prediction==i)]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[0, 1, 0], opacity = 0.5))

lines = atlas[(labels!=i) & (prediction==i)]
if len(lines)>0:
    scene.add(actor.line(lines, colors=[0, 0.5, 1], opacity = 0.5))


# lines = atlas[(labels==i)]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[0, 1, 0], opacity = 0.5))

# lines = atlas[(prediction==i)]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[1, 0, 0], opacity = 0.5))
    

# i = label_names.tolist().index('CC_ForcepsMajor')
# lines = atlas[labels==i][::30]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[1, 0.5, 0], opacity = 1))

# i = label_names.tolist().index('CC')
# lines = atlas[labels==i][::30]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[0, 1, 1], opacity = 0.5))

# i = label_names.tolist().index('FPT_L')
# lines = atlas[labels==i]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[1, .5, 0], opacity = 0.5))

# i = label_names.tolist().index('OPT_L')
# lines = atlas[labels==i]
# if len(lines)>0:
#     scene.add(actor.line(lines, colors=[.5, .5, 0], opacity = 0.5))

window.show(scene)